In [1]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import nltk
import numpy as np
import gensim
from gensim.models import Word2Vec
from nltk.data import find
from nltk.corpus import wordnet as wn
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import random

data = pd.read_csv('./all_cc_jokes.csv', sep = ',', index_col = 0, names = ['type', 'link', 'text'])
word2vec_sample = str(find('models/word2vec_sample/pruned.word2vec.txt'))
model = gensim.models.Word2Vec.load_word2vec_format(word2vec_sample, binary=False)
tfidfvectorizer = TfidfVectorizer(max_features=2000, stop_words='english')
tfidf_counts = tfidfvectorizer.fit_transform(data['text'].values.astype('U'))

In [178]:
def latent_topic(joke_index, tfidf_counts, tfidfvectorizer, model):
    """creates a latent topic for a joke using an existing word2vec model
    does so by doing a weighted avg of word2vec """
    print('\rdoing joke {}'.format(joke_index), end='')
    scores = tfidf_counts[joke_index]
    result = None
    for index in scores.indices:
        curr_word = tfidfvectorizer.get_feature_names()[index]
        try:
            if result is None:
                result = model[curr_word]
            else:
                result += model[curr_word]
        except:
            pass
    return result

def latent_topic2(joke_index, tfidf_counts, tfidfvectorizer, model):
    """creates a latent topic for a joke using an existing word2vec model
    does so by doing a weighted avg of word2vec """
    print('\rdoing joke {}'.format(joke_index), end='')
    scores = tfidf_counts[joke_index]
    result = None
    for index in scores.indices:
        curr_word = tfidfvectorizer.get_feature_names()[index]
        try:
            if result is None:
                result = model[curr_word].reshape((1,300))
            else:
                result = np.append(result,model[curr_word], axis=0)
        except:
            pass
    if result is None:
        return None
    mean = np.mean(result, axis=0)
    std = np.std(result, axis=0)
    return np.append(mean-std, mean+std, axis = 0).reshape((600))

In [384]:
generate = False
try:
    all_latent_topics = np.load('./latent_topics2.npy')
    print("successfully loaded latent topics for jokes from file")
except:
    generate = True

if generate:
    print("generating latent topics for jokes file,")
    all_latent_topics = np.zeros((data.shape[0],300))
    for i in range(data.shape[0]):
        result = latent_topic(i, tfidf_counts, tfidfvectorizer, model)
        if result is not None:
            all_latent_topics[i] = result 
    print("done! saved as latent_topics.npy")
    np.save('./latent_topics.npy', all_latent_topics)

successfully loaded latent topics for jokes from file


In [432]:
def find_most_similar(index, latent_topics, jokes):
    curr_topic = latent_topics[index]
    
    ssd = np.sum(np.abs(latent_topics - curr_topic), axis=1)
    index = 0

    while ssd[np.argsort(ssd)[index]] == 0 and index < 8:
        print ("skip, identical")
        print ('index', np.argsort(ssd)[index])
        print ('actual', np.sum(np.square(latent_topics[index] - curr_topic)))
        print ('ssd', ssd[np.argsort(ssd)[index]])
        print (jokes[np.argsort(ssd)[index]])
        index += 1
    print (np.sum(np.square(latent_topics[np.argsort(ssd)[index]] - curr_topic)))
    print(np.sum(np.square(latent_topics - curr_topic), axis=1)[np.argsort(ssd)[index]])
    print (ssd[np.argsort(ssd)[index]])
    print (ssd[np.argsort(ssd)[index+1]])
    print (ssd[np.argsort(ssd)[index-1]])

    return np.argsort(ssd)[index]

# set to random, or you can set index to a number
index=random.randint(0,data.shape[0])
# index=13929
print(index)
print (data['text'][index])
similar_index = find_most_similar(index, all_latent_topics, data['text'])
print(similar_index)
print (data['text'][similar_index])


14474
Yo' Mama is so fat, I had to take a train and two buses just to get on her good side. 
skip, identical
index 6021
actual 60362000787.7
ssd 0.0
Every once in a while, I'll be walking around, going, 'Look at me! My clothes are kind of baggy. Maybe I am losing weight.' Turns out -- just laundry time. Maybe I shouldn't wear the same jeans six days in a row. 
skip, identical
index 7797
actual 551943133514.0
ssd 0.0
Guys can always be pigs. Guys can just do anything. You see guys walking along with their pants halfway down so you can see the crack of their butt. That's the good way to find a mechanic in the city. 
skip, identical
index 4526
actual 457487705.553
ssd 0.0
I met Hillary Clinton about seven months ago. Talk about strange bedfellows' right there. And we didn't even really talk to each other -- we kinda just stood next to each other at the urinal for a couple of minutes. 
skip, identical
index 11552
actual 551943133514.0
ssd 0.0
Our Supreme Court has even ruled that forcing o

In [728]:
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn import linear_model

def guess_ratings(indices, ratings, joke_features):
    alpha = 9999999999
#     lasso =linear_model.LinearRegression()
#     lasso = Lasso(alpha=alpha, max_iter = 20000, tol=.1)
    lasso = linear_model.Ridge (alpha = .1)
#     lasso = ElasticNet(alpha=alpha, l1_ratio=0.7, max_iter=20000)
#     lasso

    y_pred_lasso = lasso.fit(joke_features[indices], ratings).predict(joke_features)
    y_pred_lasso -= np.mean(y_pred_lasso)
    y_pred_lasso *= 1.5/np.std(y_pred_lasso)
    y_pred_lasso += 2.5
    return y_pred_lasso
# add a joke's index and then its corresponding rating 
indices = [9539, 9943, 14327, 14328, 13048, 13058, 8698, 8701, 2578, 2598, 5497, 6235, 13886, 0, 15017, 13882, 10817, 13867, 14196, 13860, 13857, 10764, 13876, 10830, 10820, 13894, 13889, 13864, 13893, 14086, 13871, 13874, 12070, 13861, 13829, 13871, 12806, 12916, 14609, 14506, 14232, 13892, 13896, 13052, 13078, 4479, 10665, 3220, 5178, 4971, 10065, 14513, 4851, 14453, 14406, 14427, 221, 1399, 9912, 294, 9192, 130, 9960, 5156, 14276, 10745, 10175, 14434]
ratings = [4.5, 2, 3.3, 4, 5, 2, 3.8, 3, 2, 4, 3.5, 1, 3, 4, 3.5, 3, 0, 1, .5, 0, 2, 1, 0, 4, 4.5, 4.5, 2, 3, 3, 1, 2, 4, 1, 4, 2, 3, 4, 0, 4, 1, 2.1, 1, 2, 0, 0, 0, 5, 3, 4.5, 5, 1, 3.3, 3.4, 3, 3, 1, 0, 0, 0, 3, 2, 3, 1, 3, 2, 2, 2, 5]
indices, ratings = np.asarray(indices), np.asarray(ratings)
guesses = guess_ratings(indices, ratings, all_latent_topics)

# uncomment this to get "good jokes"
# blah = random.randint(-1000, -1)

# uncomment this to get "bad jokes"
# blah = random.randint(0, 1000)

# uncomment this to get a completely random joke
blah = random.randint(0, 15000)


joke_index = np.argsort(guesses)[blah]
while joke_index in indices:
    blah -= 1
    joke_index = np.argsort(guesses)[blah]
# joke_index = random.randint(0, 15000)
print('index: ', joke_index)
print('rating: ', guesses[joke_index])
print(data['text'][joke_index])

index:  13614
rating:  2.70885298667
A blonde was taking helicopter lessons. The instructor said, "I'll radio you every 1,000 feet to see how you're doing."
 At 1,000 feet, the instructor radioed her and said she was doing great. At 2,000 feet, he said she was still doing well.
 
 Right before she got to 3,000 feet, the propeller stopped, and she twirled to the ground. The instructor ran to where she crashed and pulled her out of the helicopter. He asked her, "What went wrong?"
 
 The blonde said, "At 2,500 feet, I started to get cold, so I turned the big fan off." 
